In [42]:
import pandas as pd
dataSet= pd.read_csv("biodegradable_a.csv", sep=",")
dataSet

,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,NaN,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,NaN,2.0,...,0.0,0.0,1.0,NaN,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,NaN,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,NaN,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,NaN,2.0,3.0,...,0.0,0.0,0.0,NaN,3.059198,0.0,7.908802,0.0,0.0,RB


### Remover os espaços em branco do DataSet:

##### KNNInputer

In [43]:
import pandas as pd
from sklearn.impute import KNNImputer

# Escolhemos 5 vizinhos de modo a obter uma valor médio para cada valor em falta 
# ou seja nem muito adaptado aos dados (overfitting) nem muito suave (underfitting)
imputer = KNNImputer(n_neighbors=5, weights="uniform");
badColumns = dataSet.columns[dataSet.isnull().any()];

print(badColumns)

dataSet[badColumns] = imputer.fit_transform(dataSet[badColumns]);

dataSet

Index(['F01', 'C', 'nCp', 'HyWi_B', 'F03_CO', 'Me', 'nCIR', 'SpMax_A', 'SdO',
       'nCrt', 'SpMax_B', 'Psi_i_A', 'nX'],
      dtype='object')


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,RB


Categorizar a variável Biodegradable em binário.

In [44]:
binario = pd.get_dummies(dataSet, columns=['Biodegradable'], prefix=['Biodegradable']);

dataSet = pd.concat([dataSet, binario['Biodegradable_RB']], axis=1)
dataSet = dataSet.drop('Biodegradable', axis=1)


dataSet


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable_RB
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,1
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,1
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,1
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,1
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,1
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,1
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,1
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,1


In [45]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()


# # Dizemos então que X sao as features e o Y é a variavel de classificação
# X = dataSet.drop('Biodegradable_RB', axis=1)
# y = dataSet['Biodegradable_RB']

# # Faz-se a divisao entre teste e treino 
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

# #Escalam-se os dados...
# scaler = StandardScaler()
# toScale = X_train.columns
# scaler.fit(X_train[toScale])

# # Escalam-se os dados de treino 
# X_train[toScale] = scaler.transform(X_train[toScale])

# # Escalam-se os dados de teste
# X_test[toScale] = scaler.transform(X_test[toScale])




### Scaling data... To see wich is the best scaler



In [58]:
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler,StandardScaler

# Método retirado do notebook: EC_TP04.ipynb -> 2022/2023
# Este método imprime um conjunto de estatísticas sobre um determinado modelo treinado e as suas previsoes.
def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    #print()
    #print("This is the Confusion Matrix")
    #print(pd.DataFrame(confusion_matrix(truth, preds)))


scalers = {
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler(),
    'MinMaxScaler': MinMaxScaler()
}

def testScalers(scalers, model, results,Xtrain,Xtest,Ytrain,Ytest):

    for scaler_name, scaler in scalers.items():
        X_train_scaled = scaler.fit_transform(Xtrain)
        X_test_scaled = scaler.transform(Xtest)
        
        model.fit(X_train_scaled, Ytrain)
        
        Ypred = model.predict(X_test_scaled)
        
        accuracy = accuracy_score(Ytest, Ypred)
        
        results[scaler_name] = accuracy

        print("Scaler: ",scaler_name,"\n");
        printClassResults(Ytest,Ypred);
        print()



##### Logistic Regression:

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,LogisticRegression(max_iter=1000,random_state=0),results,X_train,X_test,y_train,y_test);




Scaler:  StandardScaler 

The Accuracy is:  0.9608
The Precision is:  0.9634
The Recall is:  0.9914
The F1 score is:  0.9772
The Matthews correlation coefficient is:  0.8441

Scaler:  RobustScaler 

The Accuracy is:  0.9575
The Precision is:  0.9611
The Recall is:  0.9898
The F1 score is:  0.9753
The Matthews correlation coefficient is:  0.8304

Scaler:  MinMaxScaler 

The Accuracy is:  0.9416
The Precision is:  0.9425
The Recall is:  0.9914
The F1 score is:  0.9663
The Matthews correlation coefficient is:  0.7612



Tendo em conta os resultados para o Logistic Regression temos que o **Standart Scaler** obteve melhores resultados

#### Decision Trees:

In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,DecisionTreeClassifier(),results,X_train,X_test,y_train,y_test);




Scaler:  StandardScaler 

The Accuracy is:  0.9595
The Precision is:  0.9749
The Recall is:  0.9772
The F1 score is:  0.9761
The Matthews correlation coefficient is:  0.8444

Scaler:  RobustScaler 

The Accuracy is:  0.9608
The Precision is:  0.9772
The Recall is:  0.9765
The F1 score is:  0.9768
The Matthews correlation coefficient is:  0.8505

Scaler:  MinMaxScaler 

The Accuracy is:  0.9555
The Precision is:  0.9719
The Recall is:  0.9757
The F1 score is:  0.9738
The Matthews correlation coefficient is:  0.8285



Tendo em conta os resultados para as Decision Trees temos que o **Robust Scaler** obteve melhores resultados

#### Random Forests:

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,RandomForestClassifier(),results,X_train,X_test,y_train,y_test);

Scaler:  StandardScaler 

The Accuracy is:  0.9761
The Precision is:  0.9821
The Recall is:  0.9898
The F1 score is:  0.9859
The Matthews correlation coefficient is:  0.9073

Scaler:  RobustScaler 

The Accuracy is:  0.9708
The Precision is:  0.9782
The Recall is:  0.9874
The F1 score is:  0.9828
The Matthews correlation coefficient is:  0.8863

Scaler:  MinMaxScaler 

The Accuracy is:  0.9728
The Precision is:  0.9805
The Recall is:  0.9874
The F1 score is:  0.9840
The Matthews correlation coefficient is:  0.8945



Tendo em conta os resultados para o Random Forests temos que o **Standart Scaler** obteve melhores resultados

####

#### SVM:

In [62]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,SVC(),results,X_train,X_test,y_train,y_test);

Scaler:  StandardScaler 

The Accuracy is:  0.9688
The Precision is:  0.9716
The Recall is:  0.9922
The F1 score is:  0.9817
The Matthews correlation coefficient is:  0.8770

Scaler:  RobustScaler 

The Accuracy is:  0.9608
The Precision is:  0.9620
The Recall is:  0.9929
The F1 score is:  0.9772
The Matthews correlation coefficient is:  0.8440

Scaler:  MinMaxScaler 

The Accuracy is:  0.9502
The Precision is:  0.9531
The Recall is:  0.9898
The F1 score is:  0.9711
The Matthews correlation coefficient is:  0.7992



Tendo em conta os resultados para o SVM temos que o **Standart Scaler** obteve melhores resultados

### Tendo em conta que em todos os testes usando 4 modelos diferentes, o StandartScaler obteve os melhores resultados em 3 dos 4 testes, *a não ser que seja utilizado no final o modelo Decision Trees*, deve ser utilizado o método de scaling : **StandartScaler**